# Missing Values

Introduction

There are many ways data can end up with missing values. For example,

* A 2 bedroom house won't include a value for the size of a third bedroom.
* A survey respondent may choose not to share his income.
Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with missing values. So you'll need to choose one of the strategies below.

Three Approaches
1) A Simple Option: Drop Columns with Missing Values
The simplest option is to drop columns with missing values.


Unless most values in the dropped columns are missing, the model loses access to a lot of (potentially useful!) information with this approach. As an extreme example, consider a dataset with 10,000 rows, where one important column is missing a single entry. This approach would drop the column entirely!

2) A Better Option: Imputation
Imputation fills in the missing values with some number. For instance, we can fill in the mean value along each column. Also, we can fill missing values with mean values based on grouped data.

The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

3) An Extension To Imputation
Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.


In this approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries. That means that we fill missing value with mean value and we add new column that shows us in which row data was missing.

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

data=pd.read_csv('melb_data.csv')
data.head(2)


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0


In [20]:
y=data.Price

#first we take all columns beside Price
melb_pred=data.drop(['Price'], axis=1)
X = melb_pred.select_dtypes(exclude=['object'])

In [21]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [22]:
X_train.shape

(10864, 12)

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# 1. Approach - drop columns with missing values

!!!!!!!Since we are working with both training and validation sets, we are careful to drop the same columns in both DataFrames.!!!!!!!

In [24]:
# getting names of columns with missing values

cols_with_miss=[col for col in X_train.columns if X_train[col].isnull().any()]

what we did that we basically created a loop that says for columns in X_train find each that have at one or more rows NULL

In [25]:
#drop columns n training and validation data

reduced_X_train=X_train.drop(cols_with_miss, axis=1)
reduced_X_valid=X_valid.drop(cols_with_miss, axis=1)

In [26]:
print("MAE from Approach 1 (drop columns with missing values):")
print(score_dataset(reduced_X_train,reduced_X_valid,y_train,y_valid))

MAE from Approach 1 (drop columns with missing values):
183774.4028718704


# 2. Approach - Imputation

Next, we use SimpleImputer to replace missing values with the mean value along each column.

Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset). While statisticians have experimented with more complex ways to determine imputed values (such as regression imputation, for instance), the complex strategies typically give no additional benefit once you plug the results into sophisticated machine learning models.

In [27]:
from sklearn.impute import SimpleImputer

In [28]:
#imputation
my_imputer=SimpleImputer()
imputed_X_train=pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid=pd.DataFrame(my_imputer.transform(X_valid))

#imputation removed column names; puttinh them back

imputed_X_train.columns=X_train.columns
imputed_X_valid.columns=X_valid.columns

In [29]:
print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
177201.8640648012


# 3. Approach - An Extension to Imputation

In [30]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_miss:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

In [31]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
180599.43842660775


In [32]:
print(X_train.shape)

(10864, 12)


In [33]:
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
